In [4]:
# STEP 1: Install required packages
!apt-get install poppler-utils -y
!apt install tesseract-ocr -y
!pip install pdf2image pytesseract pillow python-docx opencv-python




Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import cv2
import numpy as np
import os
from google.colab import files
from docx import Document
import json

In [5]:
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]


images = convert_from_path(pdf_path)
os.makedirs("pdf_pages", exist_ok=True)
os.makedirs("extracted_images", exist_ok=True)

all_text = ""
text_data = {}
doc = Document()
doc.add_heading('Extracted Text & Images from PDF', level=1)

Saving ID1508.pdf.pdf to ID1508.pdf.pdf


In [8]:
for i, image in enumerate(images):
    image_path = f"pdf_pages/page_{i+1}.png"
    image.save(image_path, "PNG")


    text = pytesseract.image_to_string(image).strip()
    all_text += f"\n--- Page {i+1} ---\n{text}\n"
    text_data[f"Page_{i+1}"] = {"text": text}


    doc.add_heading(f'Page {i+1}', level=2)
    doc.add_paragraph(text)


    cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    img_counter = 1
    images_found = []

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 100 and h > 100:
            cropped = cv_image[y:y+h, x:x+w]
            img_name = f"extracted_images/page_{i+1}_img_{img_counter}.png"
            cv2.imwrite(img_name, cropped)
            images_found.append(img_name)
            img_counter += 1


    text_data[f"Page_{i+1}"]["images"] = images_found


with open("extracted_text.txt", "w", encoding="utf-8") as f:
    f.write(all_text)

doc.save("extracted_text.docx")

with open("extracted_text.json", "w", encoding="utf-8") as f:
    json.dump(text_data, f, indent=2)


files.download("extracted_text.txt")
files.download("extracted_text.docx")
files.download("extracted_text.json")

print("✅ All done! Text and images extracted successfully.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ All done! Text and images extracted successfully.
